In [1]:
!pip install geemap

import ee
import geemap

     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 99 kB 7.3 MB/s 
     |████████████████████████████████| 5.4 MB 42.8 MB/s 
     |████████████████████████████████| 130 kB 56.7 MB/s 
     |████████████████████████████████| 3.3 MB 50.1 MB/s 
     |████████████████████████████████| 1.7 MB 36.1 MB/s 
     |████████████████████████████████| 216 kB 38.6 MB/s 
     |████████████████████████████████| 176 kB 30.2 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 95 kB 2.6 MB/s 
     |████████████████████████████████| 8.5 MB 46.8 MB/s 
     |████████████████████████████████| 1.3 MB 48.2 MB/s 
     |████████████████████████████████| 98 kB 6.3 MB/s 
     |████████████████████████████████| 44 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 44.6 MB/s 
     |████████████████████████████████| 97 kB 2.5 MB/s 
     |████████████████████████████████| 397 kB 29.3 MB/s 
     |█████████████████████

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=gR-EYY5GtuFazsUCi69Bz-y_LGpdXVEOJeHG1GFR9Vk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgW-78NTpRIbYm0KpHdxqFDD2su-IBVFY2xKvS_aKLBKkClYqrdwBQ

Successfully saved authorization token.


In [3]:
# Takes an image collection and returns a collection with monthly means
def col2monthly(col):
    
    def monthly(m):
        return (col.filter(ee.Filter.calendarRange(y_start, y_end, 'year'))
                .filter(ee.Filter.calendarRange(m, m, 'month')).mean()
                .set('month', m)
                .set('ID', ee.Number(m).format('%02d'))
                )

    
    months = ee.List.sequence(1, 12)
    col_m = ee.ImageCollection.fromImages(months.map(monthly).flatten())
    
    return col_m

def grid2scale(img):
    return img.reproject(crs='EPSG:4326', scale=scale)

# Parameters for computation
y_start = 2001
y_end = 2020
scale = 11100


In [4]:
# Define climate variables to use for clustering
# gives mm/h
p = col2monthly(ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06").select('precipitation')).map(grid2scale)

# gives K
t = col2monthly(ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('temperature_2m')).map(grid2scale)

# Define region of interest (roi)
roi = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_co', 'BR'))

In [ ]:
p.getInfo()

In [5]:
# Let's get the monthly data and calculate the following inputs:
# annual mean monthly precipitation (P_ann),
# annual mean LST (T_ann),
# highest and lowest monthly precipitation (P_max, P_min)
# highest and lowest monthly temperature (T_max, T_min)

p_mean = p.mean().multiply(24*30).rename('p_avg')
p_max = p.max().multiply(24*30).rename('p_max')
p_min = p.min().multiply(24*30).rename('p_min')

t_mean = t.mean().subtract(273).rename('t_avg')
t_max = t.max().subtract(273).rename('t_max')
t_min = t.min().subtract(273).rename('t_min')

img_input = p_mean.addBands([p_max, p_min, t_mean, t_max, t_min]) # Lista com tudo

In [6]:
Map = geemap.Map()

# Display the mean P and T
Map.addLayer(img_input, {}, 'climate_vars')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [10]:
import geemap.colormaps as gcm

cmp = gcm.palettes
parvis = {'min': 15, 'max': 30, 'palette': cmp.viridis_r}

lyrname = 'Average monthly temperature'
Map.addLayer(t_mean, parvis, lyrname)
Map.add_colorbar(parvis, label = None, discrete=False, orientation='vertical', layer_name = lyrname)

In [11]:
# Make the training dataset.
training = img_input.sample(
    region = roi,
    scale = scale,
    numPixels = 10000
)

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(9).train(training)

# Cluster the input using the trained clusterer.
result = img_input.cluster(clusterer).clip(roi)

In [43]:
Map = geemap.Map(center=[-14, -55], zoom=4)

# Koppen climate zones
czns = (ee.FeatureCollection('projects/et-brasil/assets/Climate_Koppen_Brasil')
    .reduceToImage(properties=['GRIDCODE'], reducer=ee.Reducer.first())
    )

# Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'Climate clusters K-means')
Map.addLayer(czns.randomVisualizer(), {}, 'Koppen Climate Zones')
Map

Map(center=[-14, -55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [37]:
Map = geemap.Map(center=[-14, -55], zoom=4)

l_lyr = geemap.ee_tile_layer(result.randomVisualizer(), {}, 'Climate clusters K-means')
r_lyr = geemap.ee_tile_layer(czns.randomVisualizer(), {}, 'Koppen Climate Zones')

Map.split_map(left_layer=l_lyr, right_layer=r_lyr)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [44]:
from google.colab import output
output.enable_custom_widget_manager()

geemap.linked_maps(rows=1, cols=2, ee_objects=[result.randomVisualizer(), czns.randomVisualizer()], 
                   center=[-14, -55], zoom=4,
                   vis_params=[{}, {}], labels=['1', '2'])

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…